In [1]:
import pandas as pd
import numpy as np
import os

# Vérification de la présence des fichiers
for f in os.listdir():
    if "2022" in f and f.endswith(".csv"):
        print(f)

        

2022_23_categorielle.csv
2022_23_pour_meteo.csv
accidents_2022_2023_clean.csv
accidents_2022_2023_meteo.csv
accidents_2022_2023_quantile_clean.csv
accidents_2022_clean.csv
accidents_2022_final.csv
carcteristiques-2022.csv
lieux-2022.csv
usagers-2022.csv
vehicules-2022.csv


In [2]:
# Chargement des 4 fichiers BAAC 2022
carac_2022 = pd.read_csv("carcteristiques-2022.csv", sep=';',low_memory=False)
lieux_2022 = pd.read_csv("lieux-2022.csv", sep=';', low_memory=False)
veh_2022   = pd.read_csv("vehicules-2022.csv", sep=';', low_memory=False)
usag_2022  = pd.read_csv("usagers-2022.csv", sep=';', low_memory=False)

# Aperçu rapide des tailles et colonnes
for name, df in [("carac", carac_2022), ("lieux", lieux_2022), ("veh", veh_2022), ("usag", usag_2022)]:
    print(f"{name:8s} → {df.shape[0]:>7,} lignes × {df.shape[1]:>2} colonnes")
    print(f"Colonnes principales : {list(df.columns[:8])}")
    print("-" * 60)


carac    →  55,302 lignes × 15 colonnes
Colonnes principales : ['Accident_Id', 'jour', 'mois', 'an', 'hrmn', 'lum', 'dep', 'com']
------------------------------------------------------------
lieux    →  55,302 lignes × 18 colonnes
Colonnes principales : ['Num_Acc', 'catr', 'voie', 'v1', 'v2', 'circ', 'nbv', 'vosp']
------------------------------------------------------------
veh      →  94,493 lignes × 11 colonnes
Colonnes principales : ['Num_Acc', 'id_vehicule', 'num_veh', 'senc', 'catv', 'obs', 'obsm', 'choc']
------------------------------------------------------------
usag     → 126,662 lignes × 16 colonnes
Colonnes principales : ['Num_Acc', 'id_usager', 'id_vehicule', 'num_veh', 'place', 'catu', 'grav', 'sexe']
------------------------------------------------------------


In [3]:
#selection bonne caracteristique


import pandas as pd
import numpy as np

# 1️⃣ Renommage de la clé
carac_2022.rename(columns={'Accident_Id': 'Num_Acc'}, inplace=True)

# 2️⃣ Sélection des colonnes comme pour 2023
caract_prime_22 = carac_2022[[
    'Num_Acc', 'jour', 'mois', 'an', 'hrmn',
    'lat', 'long', 'com', 'dep',
    'lum', 'atm', 'agg', 'col'  # pas de 'adr' si absente
]].copy()

# Vérifie si 'adr' existe (en 2022 elle est souvent manquante)
if 'adr' in carac_2022.columns:
    caract_prime_22['adr'] = carac_2022['adr']

lieux_prime_22 = lieux_2022[[
    'Num_Acc', 'catr', 'circ', 'surf', 'infra', 'situ', 'plan', 'prof', 'vma'
]].copy()

vehicules_prime_22 = veh_2022[[
    'Num_Acc', 'catv', 'choc', 'manv'
]].copy()

usagers_prime_22 = usag_2022[[
    'Num_Acc', 'grav', 'sexe', 'an_nais', 'catu'
]].copy()

# 3️⃣ Vérification rapide des shapes
for name, df in [
    ("caract_prime_22", caract_prime_22),
    ("lieux_prime_22", lieux_prime_22),
    ("vehicules_prime_22", vehicules_prime_22),
    ("usagers_prime_22", usagers_prime_22)
]:
    print(f"{name:<18} → {df.shape[0]:>7,} lignes × {df.shape[1]} colonnes")

# 4️⃣ Vérifie que Num_Acc est unique dans la base principale
print("\nDoublons dans caract_prime_22 :", caract_prime_22['Num_Acc'].duplicated().sum())
print("Valeurs manquantes dans Num_Acc :", caract_prime_22['Num_Acc'].isna().sum())

# Aperçu
caract_prime_22.head(3)


caract_prime_22    →  55,302 lignes × 14 colonnes
lieux_prime_22     →  55,302 lignes × 9 colonnes
vehicules_prime_22 →  94,493 lignes × 4 colonnes
usagers_prime_22   → 126,662 lignes × 5 colonnes

Doublons dans caract_prime_22 : 0
Valeurs manquantes dans Num_Acc : 0


Num_Acc  jour  mois    an   hrmn            lat            long  \
0  202200000001    19    10  2022  16:15  44,5594200000    4,7257200000   
1  202200000002    20    10  2022  08:34  46,9258100000    6,3462000000   
2  202200000003    20    10  2022  17:15  48,4931620000   -2,7604390000   

     com dep  lum  atm  agg  col                      adr  
0  26198  26    1    1    2    3   TEIL(vieille route du)  
1  25204  25    1    1    2    3                  Miranda  
2  22360  22    1    1    2    2  ROND POINT DE BREZILLET

In [4]:
import numpy as np
import pandas as pd

# ============================================================
# 1️⃣ LIEUX : agrégation (1 ligne = 1 accident)
# Hypothèse : le lieu est identique pour toutes les lignes
# ============================================================

lieux_prime_22["vma"] = lieux_prime_22["vma"].replace(-1, np.nan)

lieux_agg_22 = lieux_prime_22.groupby("Num_Acc", as_index=False).agg({
    "catr": "first",
    "circ": "first",
    "surf": "first",
    "infra": "first",
    "situ": "first",
    "plan": "first",
    "prof": "first",
    "vma": lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan
})

print("✅ lieux_agg_22 :", lieux_agg_22.shape)


# ============================================================
# 2️⃣ VÉHICULES : regroupement EXHAUSTIF par grandes familles
# ============================================================

# Groupes couvrant 100 % des valeurs BAAC de catv
VEH_GROUPS = {
    "has_2RM":   [30, 31, 32, 33, 34, 41, 42, 43],
    "has_VL":    [3, 7, 10, 35, 36, 80],
    "has_PL":    [13, 14, 15, 16, 17, 21],
    "has_TC":    [37, 38, 39, 40],
    "has_MD":    [1, 2, 50, 60],
    "has_OTHER": [0, 20, 99]
}

vehicules_prime_22["catv"] = vehicules_prime_22["catv"].fillna(-1)

# Nombre total de véhicules
vehicules_agg_22 = (
    vehicules_prime_22
    .groupby("Num_Acc")
    .agg(nb_vehicules=("Num_Acc", "count"))
    .reset_index()
)

# Variables de présence par groupe
for col, codes in VEH_GROUPS.items():
    tmp = (
        vehicules_prime_22
        .assign(flag=lambda x: x["catv"].isin(codes).astype(int))
        .groupby("Num_Acc")["flag"]
        .max()
        .reset_index()
        .rename(columns={"flag": col})
    )
    vehicules_agg_22 = vehicules_agg_22.merge(tmp, on="Num_Acc", how="left")

vehicules_agg_22.fillna(0, inplace=True)

print("✅ vehicules_agg_22 :", vehicules_agg_22.shape)


# ============================================================
# 3️⃣ CHOC : information simple mais pertinente
# ============================================================

choc_agg_22 = (
    vehicules_prime_22
    .assign(choc_multiple=lambda x: (x["choc"] == 9).astype(int))
    .groupby("Num_Acc")["choc_multiple"]
    .max()
    .reset_index()
)

vehicules_agg_22 = vehicules_agg_22.merge(choc_agg_22, on="Num_Acc", how="left")
vehicules_agg_22["choc_multiple"].fillna(0, inplace=True)


# ============================================================
# 4️⃣ USAGERS : nettoyage âge + agrégation
# ============================================================

# Calcul âge
usagers_prime_22["age"] = 2022 - usagers_prime_22["an_nais"]

# Suppression des âges aberrants
usagers_prime_22.loc[
    (usagers_prime_22["age"] < 0) | (usagers_prime_22["age"] > 100),
    "age"
] = np.nan

usagers_agg_22 = usagers_prime_22.groupby("Num_Acc", as_index=False).agg(
    nb_usagers=("Num_Acc", "count"),
    grav_max=("grav", "max"),     # variable cible
    age_moyen=("age", "mean")
)

print("✅ usagers_agg_22 :", usagers_agg_22.shape)


# ============================================================
# 5️⃣ Vérification de cohérence avec la table principale
# ============================================================

for name, df in [
    ("lieux_agg_22", lieux_agg_22),
    ("vehicules_agg_22", vehicules_agg_22),
    ("usagers_agg_22", usagers_agg_22)
]:
    inter = len(set(df["Num_Acc"]) & set(caract_prime_22["Num_Acc"]))
    print(f"{name:<18} → correspondances : {inter:,}/{len(df):,}")

display(lieux_agg_22.head(3))
display(vehicules_agg_22.head(3))
display(usagers_agg_22.head(3))


✅ lieux_agg_22 : (55302, 9)
✅ vehicules_agg_22 : (55302, 8)
✅ usagers_agg_22 : (55302, 4)
lieux_agg_22       → correspondances : 55,302/55,302
vehicules_agg_22   → correspondances : 55,302/55,302
usagers_agg_22     → correspondances : 55,302/55,302


C:\Users\Windows\AppData\Local\Temp\ipykernel_12300\3031176143.py:79: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  vehicules_agg_22["choc_multiple"].fillna(0, inplace=True)


Num_Acc  catr  circ  surf  infra  situ  plan  prof   vma
0  202200000001     4     2     1      0     1     1     1  50.0
1  202200000002     4     2     1      0     1     1     1  50.0
2  202200000003     3    -1     1      5     1     1     1  50.0

Num_Acc  nb_vehicules  has_2RM  has_VL  has_PL  has_TC  has_MD  \
0  202200000001             2        0       1       0       0       1   
1  202200000002             2        0       1       0       0       0   
2  202200000003             2        0       1       0       0       0   

   has_OTHER  choc_multiple  
0          0              0  
1          0              0  
2          0              0

Num_Acc  nb_usagers  grav_max  age_moyen
0  202200000001           2         3       44.0
1  202200000002           2         4       43.0
2  202200000003           2         4       27.5

In [5]:
import pandas as pd
import numpy as np

print("===================================================")
print("🔍 VÉRIFICATION POST-AGRÉGATION – ANNÉE 2022")
print("===================================================")

# ====================================================
# 1️⃣ Vérification LIEUX
# ====================================================

print("\n--- LIEUX ---")

cols_lieux = ["catr", "circ", "surf", "infra", "situ", "plan", "prof", "vma"]

for col in cols_lieux:
    print(f"\n📌 {col}")
    print("Valeurs uniques (top 10) :")
    print(lieux_agg_22[col].value_counts(dropna=False).head(10))
    print("NaN :", lieux_agg_22[col].isna().sum())

# Vérification spécifique vma
print("\n⚠️ Vérification vma (valeurs extrêmes)")
print("Min vma :", lieux_agg_22["vma"].min())
print("Max vma :", lieux_agg_22["vma"].max())
print("vma > 150 :", (lieux_agg_22["vma"] > 150).sum())


# ====================================================
# 2️⃣ Vérification VÉHICULES
# ====================================================

print("\n--- VÉHICULES ---")

veh_cols = [
    "nb_vehicules",
    "has_2RM", "has_VL", "has_PL",
    "has_TC", "has_MD", "has_OTHER",
    "choc_multiple"
]

for col in veh_cols:
    print(f"\n📌 {col}")
    print(lieux_agg_22.shape[0] if col not in vehicules_agg_22.columns else "")
    print(vehicules_agg_22[col].value_counts(dropna=False).head(10))

# Vérification cohérence groupes véhicules
print("\n⚠️ Accidents sans aucun groupe véhicule identifié :")
veh_group_sum = vehicules_agg_22[
    ["has_2RM", "has_VL", "has_PL", "has_TC", "has_MD", "has_OTHER"]
].sum(axis=1)

print((veh_group_sum == 0).sum(), "accidents")


# ====================================================
# 3️⃣ Vérification USAGERS
# ====================================================

print("\n--- USAGERS ---")

# nb_usagers
print("\n📌 nb_usagers")
print(usagers_agg_22["nb_usagers"].describe())
print("nb_usagers > 20 :", (usagers_agg_22["nb_usagers"] > 20).sum())

# age_moyen
print("\n📌 age_moyen")
print(usagers_agg_22["age_moyen"].describe())
print("NaN age_moyen :", usagers_agg_22["age_moyen"].isna().sum())
print("age_moyen < 0 :", (usagers_agg_22["age_moyen"] < 0).sum())
print("age_moyen > 100 :", (usagers_agg_22["age_moyen"] > 100).sum())

# grav_max (cible)
print("\n📌 grav_max (variable cible)")
print(usagers_agg_22["grav_max"].value_counts(dropna=False))
print("NaN grav_max :", usagers_agg_22["grav_max"].isna().sum())


# ====================================================
# 4️⃣ Vérification CLÉS & COHÉRENCE GLOBALE
# ====================================================

print("\n--- COHÉRENCE GLOBALE ---")

dfs = {
    "caract": caract_prime_22,
    "lieux": lieux_agg_22,
    "vehicules": vehicules_agg_22,
    "usagers": usagers_agg_22
}

for name, df in dfs.items():
    print(f"\n📌 {name}")
    print("Lignes :", df.shape[0])
    print("Num_Acc uniques :", df["Num_Acc"].nunique())
    print("Doublons Num_Acc :", df["Num_Acc"].duplicated().sum())
    print("NaN Num_Acc :", df["Num_Acc"].isna().sum())


# ====================================================
# 5️⃣ Vérification FINALE AVANT FUSION
# ====================================================

print("\n--- INTERSECTIONS AVANT FUSION ---")

base_ids = set(caract_prime_22["Num_Acc"])

for name, df in [
    ("lieux", lieux_agg_22),
    ("vehicules", vehicules_agg_22),
    ("usagers", usagers_agg_22)
]:
    inter = len(set(df["Num_Acc"]) & base_ids)
    print(f"{name:<10} → {inter:,} / {len(base_ids):,} accidents couverts")

print("\n✅ Fin des vérifications")


🔍 VÉRIFICATION POST-AGRÉGATION – ANNÉE 2022

--- LIEUX ---

📌 catr
Valeurs uniques (top 10) :
catr
4    22671
3    21162
1     5083
2     3584
7     2090
6      394
9      262
5       56
Name: count, dtype: int64
NaN : 0

📌 circ
Valeurs uniques (top 10) :
circ
 2    34610
 1     9511
 3     7696
-1     3160
 4      325
Name: count, dtype: int64
NaN : 0

📌 surf
Valeurs uniques (top 10) :
surf
 1    46026
 2     8429
 9      331
 7      218
 3       88
 8       87
 5       51
 6       37
 4       26
-1        9
Name: count, dtype: int64
NaN : 0

📌 infra
Valeurs uniques (top 10) :
infra
 0    46386
 5     2862
 9     1970
 2      961
-1      694
 3      677
 1      656
 6      519
 8      411
 4      135
Name: count, dtype: int64
NaN : 0

📌 situ
Valeurs uniques (top 10) :
situ
 1    45401
 3     4118
 8     1890
 5     1437
 4     1326
 6      729
 2      394
-1        7
Name: count, dtype: int64
NaN : 0

📌 plan
Valeurs uniques (top 10) :
plan
 1    44357
 2     5376
 3     4784
 4      7

In [6]:
# ============================================================
# FUSION FINALE 2022 (APRÈS AGRÉGATIONS)
# ============================================================

df_2022 = (
    caract_prime_22
    .merge(lieux_agg_22, on="Num_Acc", how="left")
    .merge(vehicules_agg_22, on="Num_Acc", how="left")
    .merge(usagers_agg_22, on="Num_Acc", how="left")
)

print(f"✅ Fusion réussie : {df_2022.shape[0]} lignes, {df_2022.shape[1]} colonnes")
print(f"Accidents uniques : {df_2022['Num_Acc'].nunique()}")

display(df_2022.head(3))


# ============================================================
# SAUVEGARDE DU JEU 2022 AGRÉGÉ
# ============================================================

output_path_2022 = "./accidents_2022_finalV3.csv"
df_2022.to_csv(output_path_2022, index=False, sep=";", encoding="utf-8")

print(f"✅ Fichier 2022 enregistré : {output_path_2022}")
print(f"Lignes : {len(df_2022)} | Colonnes : {len(df_2022.columns)}")


✅ Fusion réussie : 55302 lignes, 33 colonnes
Accidents uniques : 55302


Num_Acc  jour  mois    an   hrmn            lat            long  \
0  202200000001    19    10  2022  16:15  44,5594200000    4,7257200000   
1  202200000002    20    10  2022  08:34  46,9258100000    6,3462000000   
2  202200000003    20    10  2022  17:15  48,4931620000   -2,7604390000   

     com dep  lum  ...  has_2RM  has_VL  has_PL has_TC  has_MD  has_OTHER  \
0  26198  26    1  ...        0       1       0      0       1          0   
1  25204  25    1  ...        0       1       0      0       0          0   
2  22360  22    1  ...        0       1       0      0       0          0   

   choc_multiple  nb_usagers  grav_max  age_moyen  
0              0           2         3       44.0  
1              0           2         4       43.0  
2              0           2         4       27.5  

[3 rows x 33 columns]

✅ Fichier 2022 enregistré : ./accidents_2022_finalV3.csv
Lignes : 55302 | Colonnes : 33
